In [ ]:
import re
# download data from from my Google Drive
!gdown 1ixIXQ9qceDAa4P5ENqb0ydnMXaF5lRGj

In [ ]:
# read the file in to a list of tokens
f = open('text-lines-all-new-input-modified.txt')
allutts = f.read()
allutts=re.sub('\n',' } {', allutts)
allutts=re.sub('\*[a-z]+\:','', allutts)
tokens = allutts.split()
tokens=tokens[1:100000]

In [ ]:
# Select context tokens based on frequency
from collections import Counter
counts=Counter(tokens)
# Sort the tuples. The reverse argument instructs to put most frequent first rather than last (which is the default)
so=sorted(counts.items(), key=lambda item: item[1], reverse=True)
# Extract the list of tokens, by transposing the list of lists so that there is a list of tokens a list of counts and then just selecting the former
so=list(zip(*so))[0]
# Select the firs 2500 words in the list
context_vocab_size = 2500
type_list=so[0:context_vocab_size]
token_count = len(tokens)

In [ ]:
from collections import defaultdict
type_count = len(type_list)
targets = []
positions = []
contextwordcounts = []
d = defaultdict(lambda: [0] * context_vocab_size)
window_size = 2

for i, word in enumerate(tokens):
            # Find the index in the tokenized sentence vector for the beginning of the window (the current token minus window size or zero whichever is the lower)
            begin = max(i - window_size, 0)
            # Find the index in the tokenized sentence vector for the end of the window (the current token plus window size or the length of the sentence whichever is the lower)
            end  = min(i + window_size, token_count)
            # Extract the text from beginning of window to the end
            context = tokens[begin: end + 1]
            # Remove the target word from its own window
            context.remove(tokens[i])
            # Iterate over the window for this target word
            position = 0
            for token in context:
                position += 1
                if type_list.count(token) > 0:
                  # Find the ID and hence the list index for the current token
                  token_id = type_list.index(token)
                  # Add 1 to the current context word dimension for the current target word
                  d[(word,position)][token_id] += 1


In [ ]:
# Convert to a hierachical data frame
import pandas as pd
df=pd.DataFrame(d)
df.index = type_list

In [ ]:
df